In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('analysis_avgx_2')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

In [2]:
path=f'pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

# conn

In [121]:
key2tf2ratio={}
for ens in enss:
    key2tf2ratio[(ens,'j+;conn')]={}
    key2tf2ratio[(ens,'j-;conn')]={}
    
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    
    mN_jk=ens2pars_jk_meff2st[ens][:,0]
    factor_equal=1/(-3*mN_jk/4)
    
    path=f'{basepath}/conn_2pt.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0])
        
        tf2c2pt={}
        for tf in f['data'].keys():
            t=f[f'data/{tf}'][:]
            t=yu.jackknife(np.real(t[:,:,imom]))
            tf2c2pt[int(tf)]=t

    path=f'{basepath}/conn_0,0,0,0,0,0.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0,0,0,0])
        
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            t=f[f'data/{jtf}'][:]
            t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
            c3pt=yu.jackknife(t)
            ratio=np.real(c3pt/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
            key=(ens,j)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_conn={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;conn')].keys()); tfs.sort()
    ens2tfs_conn[ens]=tfs
    print(ens,tfs)

b [8, 10, 12, 14, 16, 18, 20]
c [6, 8, 10, 12, 14, 16, 18, 20, 22]
d [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]


In [125]:
'''
Symmetrization makes the result slightly different from Christian
'''

key2bare={}

overwrite=False
symmetrizeQ=True
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    gett2=lambda t:round(t/yu.ens2a[ens]/2)*2
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]

    tfmins_2st=range(8,ens2tfs_conn[ens][-1],2); tcmins_2st=range(2,ens2tfs_conn[ens][-1]//2-1)
    
    pars_jk_meff2st=ens2pars_jk_meff2st[ens]
    fittype='2st2step_SYMshare'
    
    tf2ratio=key2tf2ratio[(ens,j)]
    if symmetrizeQ:
        tf2ratio=yu.symmetrizeRatio(tf2ratio)
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st,tcmins_2st,corrQ=False,label=f'{ens}_{j}_sum',overwrite=overwrite)
    fits_2st=yu.doFits_3pt(fittype,tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=pars_jk_meff2st,symmetrizeQ=symmetrizeQ,label=f'{ens}_{j}_2st_{symmetrizeQ}',overwrite=overwrite)
    # tfmin=gett2(8*yu.ens2a['b']); tcmin=gett(0.2)
    tfmin=gett2(0.7); tcmin=gett(0.2)
    # tfmin=gett2(1.0); tcmin=gett(0.2)
    fit_2st_MA=yu.doMA_3pt(fits_2st,fitlabels=(tfmin,tcmin))

    key2bare[(ens,j)]=fit_2st_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,None,None,fits_sum,fits_2st],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[None,None,None,fit_2st_MA],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,2,5],
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,2,5],
        'fit_2st_rainbow_midpoint:[fittype,pars_jk_meff2st]':[fittype,ens2pars_jk_meff2st[ens]],
        'xunit':yu.ens2a[ens],
    }
    return dic

js_plt=['j+;conn','j-;conn']; enss_plt=enss

for ij,j in enumerate(js_plt):
    print(f'{ij}/{len(js_plt)}',j,end='                 \r')
    keys=[(ens,j) for ens in enss_plt]

    list_dic=[createDic(key) for key in keys]

    fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','midpoint','fit_2st','fit_sum'])
    yu.addRowHeader(axs,[yu.ens2label[ens] for ens in enss_plt])
    yu.finalizePlot(f'rainbow/{j}',mkdirQ=True)
    
yu.save_pkl_reg('key2bare_conn',key2bare)

# disc

In [ ]:
ens2c2pt={}; ens2moms_2pt={}; ens2c2pt0={}; ens2Njk={}
for ens in enss:
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    path=f'{basepath}disc_2pt.h5'
    with h5py.File(path) as f:
        moms_2pt=yu.moms2list(f['moms'])
        c2pt=yu.jackknife(np.real(f['data/N_N'][:,:,:]))
        
    ens2moms_2pt[ens]=moms_2pt
    ens2c2pt[ens]=c2pt
    ens2c2pt0[ens]=c2pt[:,:,moms_2pt.index([0,0,0])]
    ens2Njk[ens]=len(c2pt)

key2tf2ratio={}
stouts=[5,7,10,15,20]
js=['j+;disc','js;disc','jc;disc']+[f'jg;stout{stout}' for stout in stouts]
for ens in enss:
    path=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/{yu.ens2full[ens]}/data_merge/disc_0,0,1,0,0,0.h5'
    with h5py.File(path) as f:
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            if j not in js:
                continue        
            key=(ens,j)
            if key not in key2tf2ratio:
                key2tf2ratio[key]={}
            c3pt=yu.jackknife(f['data'][jtf][:,:,0,projs.index('P0'),inserts.index('tz')])
            c2pt=ens2c2pt[ens][:,tf,ens2moms_2pt[ens].index([0,0,1])]
            factor=1/(1j*2*np.pi/yu.ens2NL[ens])
            ratio=np.real(c3pt/c2pt[:,None]*factor)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_disc={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;disc')].keys()); tfs.sort()
    ens2tfs_disc[ens]=tfs
    print(ens,tfs[0],tfs[-1])
    
path='pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

b 2 22
c 2 26
d 2 30


In [ ]:
selections=['bandfit_WA','const_MA','sum_MA','2st_MA']
selection2key2bare={selection:{} for selection in selections}

overwrite=False
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tfmins_1st=get_tfs(0.3,1.25)
    tcmins_1st=get_tfs(2*lat_a,0.6)
    tfmins_sum=get_tfs(0.2,1.0); tcmins_sum=[2]
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.3))
    fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_band')
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.7),tf_max=gett(1.3),tcmin=gett(0.2),corrQ=False)
    fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,corrQ=True,label=f'{ens}_{j}_const',overwrite=overwrite)
    tfmin=gett(0.6); tcmin=gett(0.3)
    fit_const_MA=yu.doMA_3pt(fits_const,fitlabels=(tfmin,tcmin))
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_sum,tcmins_sum,corrQ=False,label=f'{ens}_{j}_sum',overwrite=overwrite)
    tfmin=gett(0.35); tcmin=2
    fit_sum_MA=yu.doMA_3pt(fits_sum,fitlabels=(tfmin,tcmin))
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    selection2key2bare['const_MA'][key]=fit_const_MA[0][:,0]
    selection2key2bare['sum_MA'][key]=fit_sum_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,None],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,None],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.3),None,None],
        'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'xunit':yu.ens2a[ens],
    }
    return dic

def createDic2(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tfmins_1st=get_tfs(0.3,1.25)
    tcmins_1st=get_tfs(2*lat_a,0.6)
    tfmins_sum=get_tfs(0.2,1.0); tcmins_sum=[2]
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.3))
    fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_band')
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.7),tf_max=gett(1.3),tcmin=gett(0.2),corrQ=False)
    fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,corrQ=True,label=f'{ens}_{j}_const',overwrite=overwrite)
    tfmin=gett(0.6); tcmin=gett(0.3)
    fit_const_MA=yu.doMA_3pt(fits_const,fitlabels=(tfmin,tcmin))
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_sum,tcmins_sum,corrQ=False,label=f'{ens}_{j}_sum',overwrite=overwrite)
    tfmin=gett(0.35); tcmin=2
    fit_sum_MA=yu.doMA_3pt(fits_sum,fitlabels=(tfmin,tcmin))
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    selection2key2bare['const_MA'][key]=fit_const_MA[0][:,0]
    selection2key2bare['sum_MA'][key]=fit_sum_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,None],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,None],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.3),None,None],
        'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'xunit':yu.ens2a[ens],
    }
    return dic

# js_plt=['j+;disc']; enss_plt=['b','c','d']
# js_plt=['jg;stout10']; enss_plt=['b','c','d']
js_plt=js; enss_plt=enss

for ij,j in enumerate(js_plt):
    print(f'{ij}/{len(js_plt)}',j,end='                 \r')
    keys=[(ens,j) for ens in enss_plt]

    list_dic=[createDic2(key) if key[1].startswith('jg') else createDic(key) for key in keys]

    yu.makePlot_3pt(list_dic,shows=['rainbow', 'fit_band', 'fit_const', 'fit_sum'])
    yu.finalizePlot(f'rainbow/{j}',mkdirQ=True)

yu.save_pkl_reg('selection2key2bare_disc',selection2key2bare)

# make tftc2key2bare

In [103]:
# conn
# tf_base,tc_base=8,2
# tf_phy,tc_phy=tf_base*yu.ens2a['b'],tc_base*yu.ens2a['b']
# key2bare_conn={}
# js=['j+;conn','j-;conn']
# for ens,j in product(enss,js):
#     key=(ens,j)
#     fits=yu.getFits(f'{ens}_{j}_2st')
#     fit=yu.findFit(fits,(round(tf_phy/yu.ens2a[ens]/2)*2,round(tc_phy/yu.ens2a[ens])))
#     key2bare_conn[key]=fit[1][:,0]
key2bare_conn=yu.load_pkl_reg('key2bare_conn')
        
# disc
stouts=[5,7,10,15,20]
tftc2key2bare={}
js=[f'{j};disc' for j in ['j+','js','jc']] + [f'jg;stout{n}' for n in stouts]
# js=js[:1]

fits=yu.getFits(f'b_j+;disc_const')

tfs_phy=np.around(np.arange(0.3,1.2,0.1),2); tcs_phy=np.around(np.arange(0.1,0.6,0.1),2)

for tf_phy,tc_phy in product(tfs_phy,tcs_phy):
    key2bare=key2bare_conn.copy()
    flag=True
    for ens,j in product(enss,js):
        key=(ens,j)
        fits=yu.getFits(f'{ens}_{j}_const')
        tf,tc=round(tf_phy/yu.ens2a[ens]),round(tc_phy/yu.ens2a[ens])
        tc=max(tc,2)
        fit=yu.findFit(fits,(tf,tc))
        if fit is None:
            flag=False
            break
        key2bare[key]=fit[1][:,0]
    
    if flag:
        tftc2key2bare[(tf_phy,tc_phy)]=key2bare
        
yu.save_pkl_reg('tftc2key2bare',tftc2key2bare)

# combine

In [104]:
tftc2key2bare=yu.load_pkl_reg('tftc2key2bare')
ens2Njk={ens:tftc2key2bare[list(tftc2key2bare.keys())[0]][(ens,'j+;conn')].shape[0] for ens in enss}

In [105]:
# RCs
path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo(ens2RCs_me[ens][key],ens2RCs_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]
        
path='data_aux/RCs_pre.pkl'
with open(path,'rb') as f:
    ens2RCs_pre_me=pickle.load(f)
ens2RCs_pre={ens:{} for ens in ens2RCs_pre_me.keys()}
for ens in ens2RCs_pre_me.keys():
    for key in ens2RCs_pre_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs_pre[ens][key]=yu.jackknife_pseudo(ens2RCs_pre_me[ens][key],ens2RCs_pre_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]

In [106]:
tftcs=list(tftc2key2bare.keys()); tftcs.sort()

def run(tftc):
    key2bare=tftc2key2bare[tftc]
    yum.extendBare_avgx(key2bare)
    key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)
    key2phy_pre=yum.bareRC2phy_avgx_pre(key2bare,ens2RCs,ens2RCs_pre)
    return key2phy,key2phy_pre

tasks=tftcs
results=yu.parallelizeTasks(run,tasks,max_workers=13)

tftc2key2phy={}; tftc2key2phy_pre={}
for task,res in zip(tasks,results):
    tftc2key2phy[task],tftc2key2phy_pre[task]=res

26/26 tasks completed          


In [107]:
j2me=yu.load_pkl('pkl/lhapdf/reg_ignore/j2me_NNPDF40_nnlo_as_01180.pkl')
j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'q','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

def addexp(ax0,ax1):
    ax=ax0
    for j in ['jtot','jq','jg']:
        m,e=j2me[j]
        ax.errorbar(0.0002,m,e,color=j2color[j],mfc='white',fmt=j2fmt[j],markersize=4)
    ax=ax1
    for j in ['ju','jd','js','jc']:
        m,e=j2me[j]
        ax.errorbar(0.0002,m,e,color=j2color[j],mfc='white',fmt=j2fmt[j],markersize=4)

def addexp_span(ax0,ax1):
    ax=ax0
    for j in ['jtot','jq','jg']:
        m,e=j2me[j]
        ax.axhspan(m-e,m+e,color=j2color[j],alpha=0.4)
    ax=ax1
    for j in ['ju','jd','js','jc']:
        m,e=j2me[j]
        ax.axhspan(m-e,m+e,color=j2color[j],alpha=0.4)

In [ ]:
# for tftc in tftc2key2phy.keys():
#     print(tftc,yu.jackme_un2str(tftc2key2phy[tftc][('a=#_MA','jtot;stout10')][:,:1]))

In [108]:
tfs=list({tf for tf,tc in tftcs}); tfs.sort()
tcs=list({tc for tf,tc in tftcs}); tcs.sort()

for ce in ['MA','const','linear']:
    for nst in stouts:
        fig,axs=yu.getFigAxs(2,2,sharex=True,sharey='row')
        fig.suptitle(f'{ce}; stout{nst}')
        yu.addColHeader(axs,['pert','nonp (pre, partial)'])

        axs[0,0].set_ylim([0,1.4])
        axs[1,0].set_ylim([-0.1,0.5])

        for icol in [0,1]:
            axs[1,icol].set_xlabel(r'$t_{s,low}^{phy}$')
            dic=[tftc2key2phy,tftc2key2phy_pre][icol]
            addexp_span(axs[0,icol],axs[1,icol])
            for tftc in tftcs:
                tf,tc=tftc
                ax=axs[0,icol]
                for j in ['jtot','jq','jg']:
                    key=(f'a=#_{ce}',f'{j};stout{nst}')
                    mean,err=yu.jackme(dic[tftc][key][:,0])
                    plt_x=(tf+tcs.index(tc)*0.01); plt_y=mean; plt_yerr=err
                    ax.errorbar(plt_x,plt_y,plt_yerr,color=j2color[j],fmt=j2fmt[j])
                ax=axs[1,icol]
                for j in ['ju','jd','js','jc']:
                    key=(f'a=#_{ce}',f'{j};stout{nst}')
                    mean,err=yu.jackme(dic[tftc][key][:,0])
                    plt_x=(tf+tcs.index(tc)*0.01); plt_y=mean; plt_yerr=err
                    ax.errorbar(plt_x,plt_y,plt_yerr,color=j2color[j],fmt=j2fmt[j])
                    
        yu.finalizePlot(f'tftc_dep/{ce};stout{nst}',mkdirQ=True)

In [ ]:
for tc_phy in [0.1,0.2,0.3,0.4]:
    print([round(tc_phy/yu.ens2a[ens]) for ens in enss])

[1, 1, 2]
[3, 3, 4]
[4, 4, 5]
[5, 6, 7]


In [110]:
stouts_plt=stouts
figs=[]
for ce in ['MA','const','linear']:
    tftc=(0.6,0.3)
    list_dic=[
        {'key2phy':yum.convert_key2phy_stout(tftc2key2phy[tftc],stout),
        'key2phy_pre':yum.convert_key2phy_stout(tftc2key2phy_pre[tftc],stout)}
        for stout in stouts_plt
    ]
    fig,axs=yum.makePlot_a2dependence_avgx(list_dic,ce=ce)
    
    for i in range(len(list_dic)):
        addexp(axs[0,i],axs[1,i])
        
    fig.suptitle(f'{ce}')
    yu.addColHeader(axs,[f'stout{stout}' for stout in stouts_plt])
    yu.finalizePlot(closeQ=True)
    figs.append(fig)
    
yu.makePDF('a2dep_stout',figs)

In [111]:
ce='MA'; tftc=(0.6,0.3); nst=10

list_dic=[
    {'key2phy':yum.convert_key2phy_stout(tftc2key2phy[tftc],nst),
    'key2phy_pre':yum.convert_key2phy_stout(tftc2key2phy_pre[tftc],nst)}
]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
addexp(axs[0,0],axs[1,0])

yu.finalizePlot(f'a2dep_{ce};{tftc};{nst}')

list_dic=[
    {'key2phy':yum.convert_key2phy_stout(tftc2key2phy_pre[tftc],nst),
    'key2phy_pre':yum.convert_key2phy_stout(tftc2key2phy[tftc],nst)}
]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
addexp(axs[0,0],axs[1,0])

yu.finalizePlot(f'a2dep_{ce};{tftc};{nst}_pre')

In [112]:
ce='MA'; tftc=(0.6,0.3); nst=10

list_dic=[
    {'key2phy':yum.convert_key2phy_stout(tftc2key2phy[tftc],nst),
    'key2phy_pre':yum.convert_key2phy_stout(tftc2key2phy_pre[tftc],nst)}
]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic,noNumberQ=True,noCEonPreQ=True)

yu.finalizePlot(f'a2dep_4Report_Gregoris')

In [144]:
case='avgx'
ce='MA'; tftc=(0.6,0.3); nst=10
key2phy=yum.convert_key2phy_stout(tftc2key2phy[tftc],nst)

j='jv1'
color='r'
fmt='s'

fig,axs=yu.getFigAxs(1,1,Lrow=4,Lcol=6)
ax=axs[0,0]
ax.set_xlim([0,yum.lat_a2s_plt[-1]])
ax.set_xticks([0,0.003,0.006])
ax.set_ylim([-0.1,0.5])

m,e=j2me[j]
ax.errorbar(0.0002,m,e,color=color,mfc='white',fmt=fmt,markersize=4)

color='r'
caselabel={'avgx':r'\langle \mathrm{x} \rangle', 'B20':r'B20', 'J':r'J'}[case]
mean,err=yu.jackme(key2phy[(f'a=#_{ce}',j)])
x=yum.lat_a2s_plt; ymin=mean-err; ymax=mean+err
ax.plot(x,mean,color=color,linestyle='--',marker='')
ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)

label=rf'${caselabel}_{{u-d}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
for iens,ens in enumerate(enss):
    plt_x=yu.ens2a[ens]**2; plt_y,plt_yerr=yu.jackme(key2phy[(ens,j)])
    ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=fmt,label=label if iens==0 else None)
ax.legend()
yu.finalizePlot(f'a2dep_{ce};{tftc};{nst}_jv1')

In [143]:
def key2phy2df(key2phy):
    js=['jtot','jq','jg','ju','jd','js','jc','jv1']
    enss=['a=#_MA','d','c','b']
    columns=['a=0' if ens[:3]=='a=#' else yu.ens2label[ens] for ens in enss]
    t=[[yu.jackme_un2str(key2phy[(ens,j)][:,0] if ens[:3]=='a=#' else key2phy[(ens,j)]) for ens in enss] for j in js]
    t=pd.DataFrame(t,index=js,columns=columns)
    return t

key2phy=yum.convert_key2phy_stout(tftc2key2phy[tftc],nst)
df=key2phy2df(key2phy)
display(df)

,a=0,D96,C80,B64
jtot,0.963(73),0.99(11),0.912(61),0.885(49)
jq,0.521(46),0.641(82),0.452(45),0.528(36)
jg,0.441(45),0.346(56),0.460(32),0.358(23)
ju,0.321(16),0.355(25),0.302(17),0.336(15)
jd,0.162(12),0.192(22),0.140(13),0.158(11)
js,0.031(12),0.059(21),0.019(11),0.0298(85)
jc,0.0070(97),0.035(18),-0.0093(90),0.0034(68)
jv1,0.1585(91),0.1633(82),0.1625(82),0.178(10)
